# MLP + LSTM with MNIST
TODO: Test mlp_with_lstm on PyCharm

## Pre-requisites

Install the following Python packages

1. cloudmesh-installer
2. cloudmesh-common

In [1]:
try:
    from cloudmesh.common.StopWatch import StopWatch
except:  # noqa: E722
    ! pip install cloudmesh-common
    from cloudmesh.common.StopWatch import StopWatch

## Exporting Output Graphs

In [2]:
def save(graph, filename):
    if os.path.exists("images"):
        pass
    else:
        Shell.mkdir("images")
    plot_model(graph, to_file=f'images/{filename}.png', show_shapes=True)
    plot_model(graph, to_file=f'images/{filename}.pdf', show_shapes=True)

# Import Libraries

In [3]:
StopWatch.start("total")
StopWatch.start("import")
import os    # noqa: E402
import cpuinfo    # noqa: E402
import numpy as np    # noqa: E402
from keras.models import Sequential    # noqa: E402
from keras.layers import Dense, Activation, SimpleRNN, InputLayer, LSTM, Dropout    # noqa: E402
from keras.utils import to_categorical, plot_model    # noqa: E402
from keras.datasets import mnist    # noqa: E402
from cloudmesh.common.systeminfo import os_is_windows    # noqa: E402
from cloudmesh.common.Shell import Shell    # noqa: E402

filename = Shell.map_filename(f'~/reu2022/code/deeplearning/mnist/mnist_mlp_with_lstm.log').path
StopWatch.progress(0, filename=filename)

StopWatch.stop("import")
StopWatch.progress(20, filename=filename)

# cloudmesh status=running progress=0 pid=19696
# cloudmesh status=running progress=20 pid=19696


'# cloudmesh status=running progress=20 pid=19696'

In [4]:
StopWatch.start("data-load")

(x_train, y_train), (x_test, y_test) = mnist.load_data()

StopWatch.stop("data-load")
StopWatch.progress(40, filename=filename)

# cloudmesh status=running progress=40 pid=19696


'# cloudmesh status=running progress=40 pid=19696'

## Data Pre-Process

In [5]:
StopWatch.start("data-pre-process")

num_labels = len(np.unique(y_train))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size])
x_test = np.reshape(x_test,[-1, image_size, image_size])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

StopWatch.stop("data-pre-process")
StopWatch.progress(50, filename=filename)

# cloudmesh status=running progress=50 pid=19696


'# cloudmesh status=running progress=50 pid=19696'

## Define Model

In [6]:
StopWatch.start("compile")

input_shape = (image_size, image_size)
batch_size = 128
units = 256
dropout = 0.2

model = Sequential()
# LSTM Layers
model.add(LSTM(units=units,                      
                     input_shape=input_shape,
                     return_sequences=True))
model.add(LSTM(units=units, 
                     dropout=dropout,                      
                     return_sequences=True))
model.add(LSTM(units=units, 
                     dropout=dropout,                      
                     return_sequences=False))
# MLP Layers
model.add(Dense(units))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(units))
model.add(Activation('relu'))
model.add(Dropout(dropout))
# Softmax_layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
save(model, 'mnist_mlp_with_lstm')

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

StopWatch.stop("compile")
StopWatch.progress(60, filename=filename)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 28, 256)           291840    
                                                                 
 lstm_1 (LSTM)               (None, 28, 256)           525312    
                                                                 
 lstm_2 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               6

'# cloudmesh status=running progress=60 pid=19696'

## Train

In [7]:
StopWatch.start("train")

model.fit(x_train, y_train, epochs=30, batch_size=batch_size)

StopWatch.stop("train")
StopWatch.progress(80, filename=filename)

Epoch 1/30
 13/469 [..............................] - ETA: 3:11 - loss: 2.3022 - accuracy: 0.1232


KeyboardInterrupt



## Test

In [ ]:
StopWatch.start("test")

loss, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

StopWatch.stop("test")
StopWatch.stop("total")
StopWatch.progress(100, filename=filename)

if os_is_windows():
    user = os.environ["USERNAME"]
else:
    try:
        user = os.environ['USER']
    except:  # noqa: E722
        user = os.system('basename $HOME')

try:
    gpuname = ''
    for line in open('mlp_mnist.log', 'r'):
        if 'GPU' in line and line[-2] == ')':
            gpuname = gpuname + line[:line.find('(')] + '\n'
except:  # noqa: E722
    gpuname = cpuinfo.get_cpu_info()['brand_raw']

tag = 'mlp_mnist'

StopWatch.benchmark(tag=tag, node=gpuname, user=user)

# References

1. [Advance Deep Learning with Keras](https://github.com/PacktPublishing/Advanced-Deep-Learning-with-Keras)